cannon 3
---

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
%matplotlib inline
import tensorflow as tf
import keras

Simulation function

In [2]:
pi = np.pi

#simulates a ballistic trajectory and returns the smallest distance to the target position
def simulate_trajectory_distance(speed, azi, pitch, target, origin = np.array([0.,0.,0.]), timestep = 0.1, g = -10.):
    #get cartesian velocity from spherical coordinates
    velocity = speed*np.array([
        np.cos(azi)*np.cos(pitch),
        np.sin(azi)*np.cos(pitch),
        np.sin(pitch)
    ])
#     print(f'Starting velocity {velocity}.')
    #make acceleration vector
    acc = np.array([0.,0., g])
    #record origin and first position after 1 timestep
    history = [origin, origin+ velocity*timestep + (1/2)*acc*timestep**2]
    position = history[-1]
#     print(history)
    #record distances
    distances = [np.linalg.norm(target - origin), np.linalg.norm(target - position)]
    #while the ball is above the ground, timestep to change position
    while position[2] >= 0:
#         print(history[-1])
        position = 2*history[-1] - history[-2] + acc*(timestep**2)
        distances.append(np.linalg.norm(target - position))
        history.append(position)
    return np.array(distances).min()
    

In [3]:
distance = simulate_trajectory_distance(100, 0, pi/3, np.array([100,200,300]))
distance

221.16372929942816

In [18]:
seed = 2022
rng = np.random.default_rng(seed)

#eps == 1.1920928955078125e-07
eps = np.finfo(np.float32).eps.item()

gamma = 0.99
rounds_per_load = 5
minimum_acceptible_score = 500
scores_to_average = 10
max_speed = 100 #m/s
field_size = 2000 #m

hit_distance = 5 #5m radius of target
#miss_penalty = 500 #add this distance as a penalty for missing
num_inputs = 3 #target x, y, z
num_outputs = 3 #speed/max_speed, azimuth/(2pi), pitch/(pi)(
num_hidden = 32

#make model
inputs = keras.layers.Input(shape = (num_inputs,))
dense_1 = keras.layers.Dense(num_hidden, activation = 'relu')(inputs)
outputs = keras.layers.Dense(num_outputs)(dense_1)
model = keras.Model(inputs = inputs, outputs = outputs)

#optimizer and loss function
optimizer = tf.optimizers.Adam(learning_rate = 0.01)
loss_fn = keras.losses.Huber()

avg_recent_score = 0
all_scores = [0]
total_loss = tf.constant(0.)

# model.trainable_variables[1] = tf.convert_to

while avg_recent_score < minimum_acceptible_score:
    with tf.GradientTape() as tape:
        command_history = []
        loss_history = []
        #reload the cannon and shoot all shots
        for shot in range(rounds_per_load):
            target = field_size*rng.random(3) - field_size/2
            target[2] = abs(target[2])
            target = tf.convert_to_tensor(target)
            target = tf.expand_dims(target, 0)
            
            command = model(target)
            speed, azi, pitch = command[0]
            speed = np.float32(speed*max_speed)
            azi = np.float32(azi*2*pi)
            pitch = np.float32(pitch*pi)
            
            loss = simulate_trajectory_distance(speed, azi, pitch, target, timestep = 0.5)
            loss = loss_fn([0], [loss])
            loss_history.append(loss)
            
        total_loss = sum(loss_history)
        grads = tape.gradient(total_loss, model.trainable_variables)
        
        print('got to optimizing')
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        all_scores.append(total_loss)
        loss_history.clear()
        avg_recent_score = np.mean(all_scores[-scores_to_average:])
        if len(all_scores[:-2])%100 == 0:
            print(f'finished with trial {len(all_scores[:-2])}')

got to optimizing


ValueError: No gradients provided for any variable: (['dense_12/kernel:0', 'dense_12/bias:0', 'dense_13/kernel:0', 'dense_13/bias:0'],). Provided `grads_and_vars` is ((None, <tf.Variable 'dense_12/kernel:0' shape=(3, 32) dtype=float32, numpy=
array([[-0.15404779, -0.38550666,  0.17879567, -0.17078783,  0.10806009,
        -0.19836852, -0.23261036, -0.23141365,  0.1755592 ,  0.23261878,
         0.24854323, -0.39840996,  0.02823347,  0.07722616, -0.01893526,
        -0.24670692, -0.11922294,  0.2161121 , -0.29501492, -0.20330444,
        -0.24229674, -0.03097439,  0.41262928, -0.382739  , -0.11801398,
         0.13302818,  0.19009063, -0.25516063,  0.29355344, -0.08093429,
         0.1984612 , -0.10413104],
       [-0.37951094,  0.1212146 , -0.2365709 ,  0.3531926 , -0.03570837,
        -0.27961355, -0.29321268,  0.03928804,  0.07237238,  0.31452605,
        -0.02526978, -0.25421143, -0.16888985,  0.19756696, -0.13244075,
        -0.16993947,  0.27660987, -0.15632346, -0.22930066,  0.00

In [16]:
loss_fn()